In [1]:
from pathlib import Path
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from IPython.display import display, HTML

from collections import Counter
prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring



ignore_unrecognized_res=False
load_PDB_components=False

logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                                     ex1=None,
                                                                     ex2=None,
                                                                     # mute='all',
                                                                     ignore_unrecognized_res=ignore_unrecognized_res,
                                                                     load_PDB_components=load_PDB_components,
                                                                     ignore_waters=True)
                                 )
pyrosetta.rosetta.basic.options.set_boolean_option('run:ignore_zero_occupancy', False)
pyrosetta.rosetta.basic.options.set_boolean_option('in:auto_setup_metals', True)

In [2]:
pose = ph.parameterized_pose_from_file(pdb_filename='/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run/fragments/A71EV2A/aligned_files/Ax0310a/Ax0310a.pdb', wanted_ligands={'LIG': 'Cc1nn(C)c(C)c1NC(=O)CC#N'})

# RELAX

In [3]:
def relax(original: pyrosetta.Pose,
          constraint_weight: float=5,
          cycles: int=15,
          relax_to_start_coords:bool=True) -> pyrosetta.Pose:
    pose: pyrosetta.Pose = original.clone()
    # configure constraints
    scorefxn: pr_scoring.ScoreFunction = pyrosetta.get_fa_scorefxn()
    scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, constraint_weight)
    scorefxn.set_weight(pr_scoring.ScoreType.atom_pair_constraint, constraint_weight)
    pyrosetta.rosetta.basic.options.set_boolean_option('relax:constrain_relax_to_start_coords', relax_to_start_coords)
    pyrosetta.rosetta.basic.options.set_boolean_option('relax:coord_constrain_sidechains', relax_to_start_coords)
    # set up the relax sampler
    pyrosetta.rosetta.protocols.relax.FastRelax.register_options()
    relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)
    relax.constrain_relax_to_start_coords(relax_to_start_coords)
    relax.apply(pose)
    return pose

relax(pose)

pose.dump_pdb('/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run/fragments/Ax0310_relaxed.pdb')

True

# REMOVE LIGAND

In [4]:
from Bio.PDB import PDBParser, PDBIO, Select

class NonLigandAndZincSelect(Select):
    """Custom selection class to exclude ligand residues and zinc atoms."""
    def __init__(self, ligand_residue_names):
        self.ligand_residue_names = ligand_residue_names

    def accept_residue(self, residue):
        """Accept the residue if it's not a ligand."""
        return residue.get_resname().strip() not in self.ligand_residue_names

    def accept_atom(self, atom):
        """Accept the atom if it's not a zinc atom."""
        return atom.element != 'ZN'

def remove_ligands_and_zinc_from_pdb(input_pdb_path, output_pdb_path, ligand_residue_names):
    parser = PDBParser()
    structure = parser.get_structure("structure", input_pdb_path)

    io = PDBIO()
    io.set_structure(structure)

    # Save the structure excluding the specified ligands and zinc atoms
    io.save(output_pdb_path, NonLigandAndZincSelect(ligand_residue_names))

# Example usage
input_pdb_path = '/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run/fragments/Ax0310_relaxed.pdb'  # Path to your input PDB file
output_pdb_path = '/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run/fragments/TO_USE/Ax0310_relaxed_apo.pdb'  # Path where you want to save the PDB file without the ligand
ligand_residue_names = ['LIG', 'HETATM']  # List of residue names of the ligands to remove

remove_ligands_and_zinc_from_pdb(input_pdb_path, output_pdb_path, ligand_residue_names)